## CPSC 458 Hw2

In [1]:
import yfinance as yf
from yahoofinancials import YahooFinancials

### Income Portfolio
For our income portfolio, we are interested two major properties: stability and yield. We want this to be something we can live off of without much effort in maintaining the portfolio itself (passive income). To accomplish this, we will employ the following strategy:

1. **HOLD all assets in this portolio:** We should choose assets that we never have to sell as one of our major criteria is low maintenance.

2. **Prioritize assets with high dividend yield:** This is fairly straightforward, as we want to earn as much income via dividends per dollar invested. We don't want to overly prioritize this though, as we want to make sure the asset is healthy (recall criteria number 1).

3. **Prioritize assets with lower volatility:** For our portfolio to be a consistent income earner, we need to make sure the company is healthy and not too volatile so that we can expect a relatively reliable stream of income each quarter. If our portfolio dies one quarter, then we don't eat that quarter...

4. **Prioritize large cap assets:** Large cap assets are relatively stable most of the time. While nothing is immune to fluctuating prices (especially in the current political/economic climate), it's harder to push over giants than ants.

5. **Prioritize stocks with positive net income:** This typically implies a company is stable and has their finances together, as it allows a company to pay off debt, invest in equipment/materials, and make other investments.

#### Income Portfolio Investment Code

In [2]:
"""Returns suggested action on asset."""
def invest_income(ticker: str) -> str:
    weight = 0.5
    threshold = 0.8
    
    ticker_data = yf.Ticker(ticker)
    ticker_info = ticker_data.info
    quote_type = ticker_info.get('quoteType')
    sector = ticker_info.get('sector')
    
    if (quote_type == 'ETF'):
        weight *= 1.2
        trailing_annual_div_yield = ticker_info.get('trailingAnnualDividendYield')
        average_vol = ticker_info.get('averageVolume')
        
        if (trailing_annual_div_yield > 0.01 and average_vol > 5e6):
            weight *= 1.5
            
    elif (quote_type == 'EQUITY'):
        dividend_rate = 0 if ticker_info.get('dividendRate') is None else ticker_info.get('dividendRate')
        market_cap = ticker_info.get('marketCap')
        
        if (sector == 'Financial Services' or sector == 'Technology'):
            weight *= 1.2
        if (dividend_rate > 0.5 and market_cap > 2e8):
            weight *= 1.5
            
    beta = 1 if ticker_info.get('beta') is None else ticker_info.get('beta')
    if (beta > 1):
        weight *= 0.8
            
    cashflow = ticker_data.cashflow
    if (len(cashflow) > 0):
        net_income = cashflow.loc[['Net Income']].values[0][0]
        if (net_income is not None and net_income > 1e10):
            weight *= 1.2
    
    if (weight > threshold):
        return "hold"
    else:
        return "sell"

#### Code Breakdown
Notice that our code above handles ETFs and normal stocks (Equity) slightly differently. ETFs by default get a higher weight boost, as they are typically safer since they are a basket of stocks rather than any individual stock. Furthermore, there is an additional weight boost for ETFs if the trailing dividend yield is over 0.01 and average volume is over 5 million. The dividend yield is part of our goal criteria, and we only weight it up if average volume is high enough to screen out more volatile stocks that may pay high dividends but with less stability/consistency.

For normal stocks (Equity), we weight companies in the financial services or technology sectors higher by default. This is qualitatively built on the fact that financial service and technology companies serve as the core infrastructure for lots of other companies, and are thus more reliable. Similar to ETFs, we try to screen out volatile stocks by requiring a market cap greater than 200 million. Furthermore, we require a dividend rate of 0.5 for increasing weight for these stocks, ensuring we get decent income.

Finally, as another volatility check, we decrease weight if beta is too high, as anything over 1 indicates the asset is more volatile than the market itself. We also do a net income check on the cashflow statement in order to make sure assets with high net income make the cut. We also always choose to hold instead of just buying for short-term trading purposes, as we want this to be a low maintenance portfolio.

#### Example Assets

In [3]:
print(f"Vanguard 500 Index Fund ETF: {invest_income('VOO')}")
print(f"JPMorgan Chase & Co: {invest_income('JPM')}")
print(f"Bank of America Corp: {invest_income('BAC')}")
print(f"Intel Corporation: {invest_income('INTC')}")
print(f"Tesla Inc: {invest_income('TSLA')}")
print(f"BioNano Genomics Inc: {invest_income('BNGO')}")
print(f"Roku, Inc: {invest_income('ROKU')}")
print(f"FuelCell Energy Inc: {invest_income('FCEL')}")

Vanguard 500 Index Fund ETF: hold
JPMorgan Chase & Co: hold
Bank of America Corp: hold
Intel Corporation: hold
Tesla Inc: sell
BioNano Genomics Inc: sell
Roku, Inc: sell
FuelCell Energy Inc: sell


Notice how for the assets above, more stable high-dividend assets receive a HOLD action whereas more volatile risky assets receive a SELL action.

### ESG Portfolio
For our ESG portfolio, our strategy is to prioritize ESG scores in our asset selections while still looking at a few simple key indicators of company health. Namely market cap and volume. This simple double-screening method allows us to ensure that we are picking high ESG stocks while avoiding super risky assets that may destroy our portfolio. We will also more heavily positively weight some sectors that are traditionally more ESG-unfriendly, (ex. packaging, energy) as any companies in these sectors with high ESG scores are likely improving the sector (ex. renewable energy companies helping move systems off of petroleum).

In [4]:
"""Returns suggested action on asset."""
def invest_esg(ticker: str) -> str:
    weight = 0.5
    buy_threshold = 0.7
    hold_threshold = 0.85
    
    ticker_data = yf.Ticker(ticker)
    ticker_info = ticker_data.info
    quote_type = ticker_info.get('quoteType')
    sector = ticker_info.get('sector')
    
    sustainability = ticker_data.sustainability
    has_sustainability = sustainability is not None
    
    if not has_sustainability:
        return "sell"

    sector_set = { 'Basic Materials', 'Industrials', 'Technology' }
    esg_score = sustainability.loc[['totalEsg']].values[0][0]
    env_score = sustainability.loc[['environmentScore']].values[0][0]
    
    if esg_score > 5:
        weight *= 1.2
        
        if sector in sector_set:
            weight *= 1.4
            
        market_cap = ticker_info.get('marketCap')
        average_vol = ticker_info.get('averageVolume')
        
        if (market_cap > 2e8 and average_vol > 5e6):
            weight *= 1.5
            
    if (weight > hold_threshold):
        return "hold"
    elif (weight > buy_threshold):
        return "buy"
    else:
        return "sell"

#### Code Breakdown
In our code above, note we screen out stocks with no sustainability report, as we can't really determine value from an ESG perspective without this. From there, we choose stocks with a total ESG score greater than 5, and then add weight for assets in the materials, industrials, and technology sectors (recall that this might be a signal for companies introducing more sustainable practices in traditionally ESG-unfriendly sectors, so we want to give more kudos here). From there, we add a 50% weight boost for assets with a market cap greater than 20 million and average volume greater than 5 million. This is because we want to hold stocks with especially good financials given impressive ESG scores.

#### Example Assets

In [5]:
print(f"Accenture: {invest_esg('ACN')}")
print(f"Tesla, Inc: {invest_esg('TSLA')}")
print(f"Xylem Inc: {invest_esg('XYL')}")
print(f"Salesforce: {invest_esg('CRM')}")

print(f"FuelCell Energy Inc: {invest_esg('FCEL')}")
print(f"Vale S.A.: {invest_esg('VALE')}")
print(f"Fastly Inc: {invest_esg('FSLY')}")

Accenture: buy
Tesla, Inc: hold
Xylem Inc: buy
Salesforce: hold
FuelCell Energy Inc: sell
Vale S.A.: sell
Fastly Inc: sell


We see from the example assets above that we get a decent balance of ESG and profitabiliy here. Strong assets like ACN, TSLA, XYL, and CRM make the buy/hold cut whereas other more volatile and less ESG-friendly stocks like VALE don't make the cut.

### Growth Portfolio
For our growth portfolio, we will mainly focus on revenue growth and volume. We will not focus on market cap as much since companies with high growth potential do not necessarily have high market caps already (there needs to be room to grow). Furthermore, we will make sure that stocks are at least 10% under their target low price.

In addition, we will focus on two sectors: technology and diagnostics. This is because these are two growth-heavy sectors that have caught the interest of a lot of growth funds (ex. ARK Innovation ETF). It is also a good time to buy these assets right now considering current market prices.

In [11]:
"""Returns suggested action on asset."""
def invest_growth(ticker: str) -> str:
    weight = 0.5
    buy_threshold = 0.7
    hold_threshold = 0.85
    
    ticker_data = yf.Ticker(ticker)
    ticker_info = ticker_data.info
    quote_type = ticker_info.get('quoteType')
    sector = ticker_info.get('sector')
    
    sector_set = { 'Diagnostics & Research', 'Technology' }
    if (sector not in sector_set):
        weight *= 0.75
    else:
        weight *= 1.5
        
    average_vol = ticker_info.get('averageVolume')
    if (average_vol > 5e6):
        weight *= 1.5
        
    current_price = ticker_info.get('currentPrice')
    if current_price is not None:
        target_low_price = ticker_info.get('targetLowPrice')
        percent_diff = (target_low_price - current_price) / current_price
        if percent_diff > 0.10:
            weight *= 1.6
        
    rev_growth = ticker_info.get('revenueGrowth')
    if rev_growth is not None:
        if rev_growth > 0.3:
            weight *= 1.5
    
    if (weight > hold_threshold):
        return "hold"
    elif (weight > buy_threshold):
        return "buy"
    else:
        return "sell"

#### Example Assets

In [13]:
print(f"Fastly Inc: {invest_growth('FSLY')}")
print(f"Datadog Inc: {invest_growth('DDOG')}")
print(f"Unity Software Inc: {invest_growth('U')}")
print(f"Meta Platforms Inc: {invest_growth('FB')}")

print(f"Deere & Company: {invest_growth('DE')}")
print(f"Walmart: {invest_growth('WMT')}")
print(f"Mastercard Inc: {invest_growth('MA')}")

Fastly Inc: hold
Datadog Inc: hold
Unity Software Inc: hold
Meta Platforms Inc: hold
Deere & Company: sell
Walmart: sell
Mastercard Inc: sell


From the example assets above, we see that small to mid-cap stocks meet the buy/hold criteria more frequently than larger more established companies like Meta and Walmart. This makes sense since we are looking for pretty aggressive growth with this strategy, which means larger assets that have already peaked in growth rate might not make the cut.

### Invest Implementation

My invest implementation is a simple wrapper around the three strategies discussed in the portfolio types above. By default, if no client type is passed then it uses the income portfolio investment strategy (as it is a much more safe default than the others).

In [14]:
def invest(ticker: str, client=False) -> str:
    if (client == 'income'):
        return invest_income(ticker)
    elif (client == 'growth'):
        return invest_growth(ticker)
    elif (client == 'esg'):
        return invest_esg(ticker)
    return invest_income(ticker)